# SQL function creation

This notebook stores the code that creates the SQL functions that are then used by the API code. Needs to be run only once (and re-run if any updates of course!).

**Important**:In order to keep the list clean, please prefix any new functions with `api_`

In [1]:
import psycopg2
import configparser

config = configparser.ConfigParser()
config.read("../../config.ini")    
db_params = dict(config['DB'])
from sqlalchemy import create_engine

def execute_sql(SQL):
        with psycopg2.connect(**db_params) as conn:
            with conn.cursor() as cur:
                cur.execute(SQL)        


def get_alchemy_engine():
    conn_string = 'postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**db_params)
    return create_engine(conn_string, echo=False)

### Getting POIs for a given city

API endpoint: /pois

In [2]:
SQL = """

CREATE OR REPLACE FUNCTION api_get_pois_for_city(
    city_id int, poi_category character)
    RETURNS TABLE
            (
                id          int,
                h3id    char,
                name    varchar,
                lat         real,
                long        real,
                category    varchar
            )
    LANGUAGE plpgsql
AS
$poiforcity$
BEGIN    
    RETURN QUERY
        SELECT pois.poiid, pois.h3id, pois.name, pois.lat, pois.long, pois.category 
            FROM pois 
            JOIN cityh3map ON cityh3map.h3id = pois.h3id
		    JOIN cities ON cities.cityid = cityh3map.cityid
		    WHERE cities.cityID = city_id AND pois.category = poi_category;
END;
$poiforcity$;

"""

execute_sql(SQL)

### Demographics for a city

First, we create a precomputed table where all population data is allocated to individual H3 grid cells.

In [5]:
create_sql = """

DROP TABLE IF EXISTS h3demographics;

CREATE TABLE public.h3demographics
(
    cityid bigint,
    categorytype text,
    groupname text,
    h3id character(15),
    population double precision,
    id bigserial,
    CONSTRAINT h3demographics_id PRIMARY KEY (id),
    CONSTRAINT unique_key UNIQUE (cityid, categorytype, groupname, h3id)
);

CREATE INDEX IF NOT EXISTS h3id_cityid ON public.h3demographics (cityid, h3id);

CREATE INDEX IF NOT EXISTS h3demographics_h3index ON public.h3demographics (h3id);
"""

execute_sql(create_sql)

Then, we populate it using the demographics data currently available in the database. This takes a moment (~30s currently).

In [ ]:
SQL = """
WITH 
     censush3_counts AS (
		SELECT 
		 	cities.cityID, 
		 	censush3map.censusblockgroupid, 
		 	count(cityh3map.h3id) as hexagon_count
		FROM cityh3map
		JOIN cities ON cities.cityID = cityh3map.cityID
		JOIN censush3map ON censush3map.h3id = cityh3map.h3id			
		GROUP BY censush3map.censusblockgroupid, cities.cityID
	),

	--- Then, find associated population totals for 
	--- the entire census block for each H3 hexagon in the city

	demographic_totals AS (
		SELECT 
			cities.cityID, 
			demographics.categorytype, 
			censush3map.censusblockgroupid, 
			cityh3map.h3id, 
			demographics.groupname, 
			demographics.total
		FROM cityh3map
		JOIN cities 
			ON cities.cityID = cityh3map.cityID
		JOIN censush3map 
			ON censush3map.h3id = cityh3map.h3id
		JOIN demographics 
			ON censush3map.censusblockgroupid = demographics.censusblockgroupid        
	)

	--- Finally, join the above two tables together, and split the population equally among all hexagons
    --- Take the result and insert it into the h3demographics table

	INSERT INTO h3demographics (cityid, categorytype, groupname, h3id, population)
        SELECT 
            d.cityID,
            d.categorytype,
            d.groupname, 
            d.h3id, 		
            d.total::float / c.hexagon_count::float as population		
            FROM demographic_totals as d
            JOIN censush3_counts as c 
                ON 	d.censusblockgroupid = c.censusblockgroupid AND
                    d.cityID = c.cityID

"""

execute_sql(SQL)

Finally, we create a function that retrieves the information from the table.

In [7]:
SQL = """
-- Demographics for a city

CREATE OR REPLACE FUNCTION api_get_demographics_for_city(
    in_cityid integer, in_categorytype character)
    RETURNS TABLE
            (
                h3id    char,
                groupname   text,
                population   float
            )
    LANGUAGE plpgsql
AS
$demographicsforcity$
BEGIN
    
    RETURN QUERY
        SELECT h3demographics.h3id, h3demographics.groupname, h3demographics.population from h3demographics 
        WHERE cityid = in_cityid and categorytype = in_categorytype;

END;
$demographicsforcity$;

-- SELECT * FROM api_get_demographics_for_city(1, 'Race'); 
"""

execute_sql(SQL)

### Getting catchment area statistics for a given catchment area (by ID)
We'll use precomputed tables, too, as it makes it faster to get accessibility statistics afterwards.

In [3]:
create_sql = """

DROP TABLE IF EXISTS catchment_stats;

CREATE TABLE public.catchment_stats
(
    catchmentid bigint,    
    categorytype text,
    groupname text,    
    population double precision,
    id bigserial,
    CONSTRAINT catchment_stats_id PRIMARY KEY (id),
    CONSTRAINT catchment_stats_unique_key UNIQUE (catchmentid, categorytype, groupname)
);

CREATE INDEX IF NOT EXISTS catchment_stats_index ON public.catchment_stats (catchmentid, categorytype);
"""

execute_sql(create_sql)

Populating the table

In [ ]:
SQL = """
WITH all_h3_ids as (
	SELECT 		
		h3demographics.categorytype,
		h3demographics.h3id, 
		h3demographics.groupname, 
		h3demographics.population 
	FROM h3demographics
),

catchmenth3_ids AS (
	SELECT 
		catchments.catchmentid,
		catchmenth3map.h3id
	FROM catchments
	JOIN catchmenth3map ON catchmenth3map.catchmentid = catchments.catchmentid
)

INSERT INTO catchment_stats (catchmentid, categorytype, groupname, population)
	SELECT c.catchmentid, h.categorytype, h.groupname, SUM(h.population)
		FROM all_h3_ids as h
		JOIN catchmenth3_ids as c ON c.h3id = h.h3id
		GROUP BY c.catchmentid, h.categorytype, h.groupname
"""

execute_sql(SQL)

Defining the function

In [6]:
SQL = """
-- Demographics for a catchment area

CREATE OR REPLACE FUNCTION api_get_demographics_for_catchment(    
    in_categorytype character,
    in_catchment_id integer
)
    RETURNS TABLE
            (                
                groupname   text,
                population   float
            )
    LANGUAGE plpgsql
AS
$demographicsforarea$
BEGIN
    
    RETURN QUERY
        
        SELECT catchment_stats.groupname, catchment_stats.population 
            FROM catchment_stats
            WHERE categorytype = in_categorytype AND catchmentid = in_catchment_id;

END;
$demographicsforarea$;

-- SELECT * FROM api_get_demographics_for_catchment('Race', 1); 
"""

execute_sql(SQL)

# Calculating accessibility statistics based on 2FSCA method
First, we create the table

In [13]:
create_sql = """

DROP TABLE IF EXISTS accessibility_stats;

CREATE TABLE public.accessibility_stats
(
    id bigserial,
    cityid bigint,    
    categorytype text,
    groupname text,   
    poi_category varchar(50),
    timeofday varchar(50),
    h3id char(15),
    accessibility double precision,    
    CONSTRAINT acc_stats_id PRIMARY KEY (id),
    CONSTRAINT acc_stats_unique_key UNIQUE (cityid, categorytype, groupname, poi_category, timeofday, h3id)
);

CREATE INDEX IF NOT EXISTS acc_stats_agg_index ON public.accessibility_stats (cityid, categorytype, poi_category, timeofday, groupname);
CREATE INDEX IF NOT EXISTS acc_stats_h3index ON public.accessibility_stats (h3id);
"""

execute_sql(create_sql)

Then, we populate it. This runs a while - around 5 minutes. For testing, add a WHERE condition for specific POI type / H3 index / etc (then it's super fast)

In [14]:
SQL = """
    --- Step 1: for each POI get :
        -- its POI category
        -- associated catchment ID (one catchment may be shared by multiple POIs)
        -- catchment metadata (time of day)
        --- 1 / (underlying population in the catchment area) 

    -- To avoid the problem of e.g. Native Hawaiians competing for POIs only with Native Hawaiians (which is what happens if we group by groupname), we ensure that we divide by total
    -- population within a catchment, across groupnames, as opposed to on a per-groupname basis

    WITH catchmentid_to_total_population AS (
        SELECT catchmentid, SUM(population) AS pop_in_catchment
        FROM catchment_stats
        WHERE categorytype = 'Race' -- we can use any category, the population totals are the same for each catchment regardless of category
        GROUP BY catchmentid
    ),
         step1 AS (
             SELECT pois.category as poi_category,
                    catchments.timeofday,
                    catchment_stats.catchmentid,
                    catchment_stats.categorytype,
                    catchment_stats.groupname,
                    CASE
                        WHEN population = 0
                            THEN 0
                        ELSE 1 / catchmentid_to_total_population.pop_in_catchment
                        END       AS ratio
             FROM pois
                      JOIN catchments
                           ON pois.h3id = catchments.originh3id
                      JOIN catchment_stats
                           ON catchments.catchmentid = catchment_stats.catchmentid
                      JOIN catchmentid_to_total_population
                           ON catchmentid_to_total_population.catchmentid = catchments.catchmentid
         ),

    --- Data collection for step 2. We need to:
         ---  Get all h3ids in a city and their associated population attributes
         --- For each h3id, get the catchment IDs that cover those cells
         --- Note that some cells may not have a single catchment area covering them
         --- But we still need to make sure they are captured for statistics purposes

         h3_coverage as (
             SELECT h3demographics.h3id,
                    h3demographics.cityid,
                    h3demographics.categorytype,
                    h3demographics.groupname,
                    catchmenth3map.catchmentid
             FROM h3demographics
                      LEFT JOIN catchmenth3map
                                ON catchmenth3map.h3id = h3demographics.h3id
         )
    INSERT
    INTO accessibility_stats (h3id, cityid, categorytype, groupname, poi_category, timeofday, accessibility)
    SELECT h3_coverage.h3id,
           h3_coverage.cityid,
           h3_coverage.categorytype,
           h3_coverage.groupname,
           step1.poi_category,
           step1.timeofday,
           SUM(step1.ratio) as accessibility
    FROM h3_coverage
             LEFT JOIN step1
                       ON
                           h3_coverage.catchmentid = step1.catchmentid
    WHERE h3_coverage.categorytype = step1.categorytype
      AND h3_coverage.groupname = step1.groupname
    GROUP BY h3_coverage.h3id,
             h3_coverage.cityid,
             h3_coverage.categorytype,
             h3_coverage.groupname,
             step1.poi_category,
             step1.timeofday;
"""

execute_sql(SQL)

## Testing!
Statistics for Atlanta, hospitals, based on race. Looks as expected... note that cinemas/theaters and clinics/hospitals have lower metric values compared to e.g. schools or restaurants. This intuitively makes sense, given there's many fewer of the former compared to the latter.

In [16]:
sql_query = """	
    SELECT a.cityid,
           h.groupname,
           h.categorytype,
           a.poi_category,
           10000 * SUM(a.accessibility * h.population) / SUM(h.population) as metric
    FROM accessibility_stats as a
             JOIN h3demographics as h
                  ON h.h3id = a.h3id
    WHERE h.groupname = a.groupname
      AND h.categorytype = a.categorytype
      AND h.cityid = a.cityid
      AND h.categorytype = 'Race'
      and a.cityid = 1
    GROUP BY a.cityid,
             h.groupname,
             h.categorytype,
             a.poi_category
    ORDER BY poi_category, metric DESC;
	"""

import pandas as pd

pd.read_sql_query(sql_query, get_alchemy_engine())

,cityid,groupname,categorytype,poi_category,metric
0,1,Some other race,Race,Cinemas and Theaters,2.420624
1,1,Black or African American,Race,Cinemas and Theaters,1.748161
2,1,Two or more races,Race,Cinemas and Theaters,1.547574
3,1,White,Race,Cinemas and Theaters,1.544567
4,1,Asian,Race,Cinemas and Theaters,1.497059
5,1,American Indian and Alaska Native,Race,Cinemas and Theaters,1.142523
6,1,Native Hawaiian and Other Pacific Islander,Race,Cinemas and Theaters,1.108047
7,1,Some other race,Race,Clinics and Hospitals,1.977093
8,1,Native Hawaiian and Other Pacific Islander,Race,Clinics and Hospitals,1.799195
9,1,American Indian and Alaska Native,Race,Clinics and Hospitals,1.735990
